#说明：
#1  这个代码运行在第三方策略回测平台 优矿上面
#2  运行这个代码之前请把数据文件（newsNLP-buySignal-sentiment.csv）上传到回测平台上
#3  newsNLP-buySignal-sentiment.csv--这个文件是新闻的多空情感之大于0.7，与个股相关度大于0.6的财经新闻数据

#示例策略 只要新闻情感值大于0.7，相关度大于0.6,直接在新闻发布之后5分钟买入该股票。回测显示基本和沪深300走势一致。下面考虑每隔一个月统计所以情感值大于0.7的股票，按照新闻的热度进行排序，买入排名靠前的前5只股票，一个调仓一次。
#在notebook中需要选策略，不能选择“代码”
import numpy as np      #导入pyton计算模块
from datetime import datetime
from datetime import timedelta
import pandas as pd
#https://uqer.io/help/introduction/#支持哪些Python类库

print "code begin."
start = '2012-06-01' #回测开始时间
end = '2017-06-01'  # 回测结束时间

# 得到股票池
#常见股票板块 https://uqer.io/help/research/#常见股票板块
universe = set_universe('ZZ500') # SH50 HS300 ZZ500 ZXB A--全A股  如果Universe设置为全A 会带来回测莫名其妙停止的问题
benchmark = 'HS300' #'000001.ZICN'  #'HS300'  https://uqer.io/help/research/#常见指数
freq = 'd'                                 # 'd'表示使用日频率回测，'m'表示使用分钟频率回测
refresh_rate = 1                        # 执行handle_data的时间间隔
#Warning: refresh_rate的值仅作用于分钟线。若想对日线进行控制，请使用如下定义: refresh_rate = (日线refresh_rate， 分钟线refresh_rate)
commission = Commission(buycost=0.000, sellcost=0.000, unit='perValue')

#https://uqer.io/help/strategyManualV3/#策略回测基本流程
accounts = {
    'fantasy_account': AccountConfig(account_type='security', capital_base=10000000,commission=commission)
}

print "before inital."
#context： 策略运行环境。可以用于存储自定义的全局变量。https://uqer.io/help/strategyManualV3/#initialize
#context 就是以前的 account
def initialize(context):                   # 初始化策略运行环境
    print "initialize begin."
    # https://uqer.io/help/research/#常见交易所列表    证券交易所	600000.XSHG  证券交易所	000001.XSHE
    #context.buySetID = '600519.XSHG'             #茅台
    #timeStr='2017-06-01 11:01:28'
    #timeTuple = datetime.strptime(timeStr, '%Y-%m-%d %H:%M:%S')
    #context.buySignalDf = pd.read_csv('newsNLP-buySignal-sentiment-eventPerformance.csv', encoding='utf-8',parse_dates=True,dtype={"stockId":str})
    context.buySignalDf = pd.read_csv('newsNLP-buySignal-sentiment.csv', encoding='utf-8',parse_dates=True,dtype={"stockId":str})
    # time1=buySignalDf.iloc[0,0]
    # time1=datetime.strptime(time1, '%Y-%m-%d %H:%M:%S')
    # stockID1=buySignalDf.iloc[0,1]
    # print time1.strftime('%Y-%m-%d %H:%M:%S')
    # print stockID1
    # context.buyTime=timeTuple
    #context.sellSignalList=[]
    context.firstRun=True
    context.lastBuyDate=0
    context.df_index=0
    context.df_index_full=False
    print "initial ok."

def handle_data(context):
    #获取当前的时间  https://uqer.io/help/strategyManualV3/#current_date
    # context.current_date  类型:datetime
    # minuteStr=context.current_minute #字符串  '09:30'
    # timeStr2=context.current_date.strftime("%Y-%m-%d")+" "+minuteStr+":00"
    # currentTime=datetime.strptime(timeStr2, '%Y-%m-%d %H:%M:%S')
    currentTime=context.current_date
    # currentTime=context.current_date
    #print currentTime
    holdDays=30 #30
    account = context.get_account('fantasy_account')
    df_index=context.df_index
    if context.firstRun or (not context.firstRun and (currentTime>=context.lastBuyDate+ timedelta(days=holdDays))):
        context.lastBuyDate=currentTime
        #统计30天前到现在的最热新闻股票
        aMonthAgo=currentTime - timedelta(days = holdDays) 
        lenOfDataFrame=len(context.buySignalDf.index)
        hotNewsCountDic={}
        publishTime=context.buySignalDf.iloc[df_index][0]
        publishTime=datetime.strptime(publishTime, '%Y-%m-%d %H:%M:%S')
        if context.firstRun:
            context.firstRun=False
            while publishTime<aMonthAgo:
                df_index+=1
                publishTime=context.buySignalDf.iloc[df_index][0]
                publishTime=datetime.strptime(publishTime, '%Y-%m-%d %H:%M:%S')
        while df_index<lenOfDataFrame:
            publishTime=context.buySignalDf.iloc[df_index][0]
            publishTime=datetime.strptime(publishTime, '%Y-%m-%d %H:%M:%S')
            if publishTime<currentTime:
                buySecID=context.buySignalDf.iloc[df_index][1]
                if buySecID[0]=="6":
                    buySecID=buySecID+".XSHG"
                else:
                    buySecID=buySecID+".XSHE"  #深圳股票
                if buySecID in hotNewsCountDic:
                    hotNewsCountDic[buySecID]+=1
                else:
                    hotNewsCountDic[buySecID]=1
            else:#publisTime>=currentTime:
                break
            #loop in while
            df_index+=1
        #开始统计
        context.df_index=df_index
        List1=sorted(hotNewsCountDic.items(),key=lambda a_tuple:a_tuple[1],reverse=True)
        # print "hot news rank:",
        # print List1
        #开始买入
        cash=account.cash
        cash=cash/5
        buyIndex=0
        buyCount=0
        while buyCount < 5:
            buySecID=List1[buyIndex][0]
            if buySecID in context.get_universe():
                currentPrice=context.history(symbol=buySecID, attribute='closePrice', time_range=1, freq='1d', style='ast', rtype='frame')  #style='sat'   '600519.XSHG'  freq='1m',
                currentPrice=currentPrice['closePrice']
                # print currentPrice
                try:
                    currentPrice=currentPrice.iloc[0,0] #则会显示第一行第一列的数据;(切片后面的值取不到)
                    # print currentPrice
                    if currentPrice>1:
                        amount=(cash)/currentPrice
                        account.order(buySecID, amount)
                        buyCount+=1
                except:
                    print "error at get current stock price."
            buyIndex+=1            
                          
            
    #开始处理卖出信号
    if (not context.firstRun and (currentTime>=context.lastBuyDate+ timedelta(days=holdDays-1))):
        # print "selling:",
        # print account.get_positions()
        for stockId in account.get_positions():
            if stockId in context.get_universe():
                order_to(stockId, 0)
            




In [1]:
import pandas as pd
import numpy as np
from tqdm import *
pd.set_option('display.max_rows',None)
pd.set_option('display.max_columns',None)
import tushare as ts
pro = ts.pro_api('6792f89c67a1f228aaf8aa19e6b0ca381504c16d8d5b7f75d76427a5')

In [15]:
df = pro.news(src='10jqka', start_date='2021-04-09 00:00:00', end_date='2021-04-09 23:59:00')

In [18]:
df.to_csv('./xinwenlianbo_0409.csv')

In [17]:
df = pro.cctv_news(date='20210409')

In [19]:
df = pro.major_news(src='', start_date='2021-04-09 00:00:00', end_date='2021-04-09 23:59:00', fields='title,content')

In [23]:
df.content[3]

'投资研报\n【机构调仓】朱少醒、劳杰男、袁芳等大佬调仓踪迹泄露！4大牛散聚集1股(名单)\n【硬核研报】一季度业绩暴增110%，全球市占率6%，超稀缺电子元件龙头走向全球\n【超级大单】妖股怒抓4连板！三大顶级游资激战(兼谈“面板双雄”股价为何大分化)\n【机构调研】林利军、王亚伟旗下公司出手调研(附剖析A股“气宗”与“剑宗”)\n天天财经独家，速关注\n]article_adlist-->\nTCL家电集团正式进入奥马电器董事会。4月9日晚间，奥马电器公告，根据股东大会表决结果，徐荦荦和胡殿谦当选为公司第四届非独立董事。\n自今年1月起，TCL家电集团开始通过在二级市场增持及竞拍，成为奥马电器第一大股东。早在2月初，TCL家电集团就提请增补徐荦荦、胡殿谦两人为非独立董事候选人，但曾被奥马电器董事会投票否决。\n截至3月30日，TCL家电集团持有奥马电器股本比例已达到21.92%。随着徐荦荦和胡殿谦成为奥马电器非独立董事，TCL家电集团与奥马电器的攻防战暂告一段落。但今年3月，TCL创始人李东生表示，奥马在冰箱产业中是一个很有竞争力的公司。未来12个月不排除继续增持的可能。\n双方拉锯战\n4月9日，奥马电器召开了2021年第二次临时股东大会。奥马电器董事长兼董秘赵国栋主持会议。\n根据4月9日股东大会表决结果，徐荦荦和胡殿谦当选为公司第四届非独立董事，任期自本次股东大会决议通过之日起至公司第四届董事会届满时止。\n(image_address="//n.sinaimg.cn/spider202149/189/w1080h709/20210409/50e0-knqqqmu2056400.jpg")\n来源：公司公告\n]article_adlist-->\n徐荦荦和胡殿谦是TCL家电集团在3月19日向奥马电器提案增补的第四届董事会非独立董事人选。如今，TCL家电集团正式进入奥马电器董事会。\n但过程一波三折，双方之间曾上演拉锯战。今年初，TCL家电集团开始买入奥马电器股票。1月28日晚，奥马电器公告称，1月8日至1月27日期间，TCL家电集团与一致行动人中新融泽通过二级市场合计增持公司股份5420.55万股，占公司总股本的5%。此后，TCL家电集团在二级市场不断增持，并且通过竞拍司法拍卖股票，成为奥马电器第一大股东。但奥马电器似乎并不欢迎TCL的突然“闯入”。\n

In [10]:
text = df.content[90]
text

'【香港交易所：一季度32只新股集资1328亿港元 同比增逾8倍】香港交易所消息，香港IPO市场持续活跃。今年一季度共迎来32只新股在港上市，共集资1328亿港元，较去年同期大增逾8倍。同时，香港证券市场一季度平均每日成交金额达2,244亿港元，较去年同期上升86%。（e公司）'

In [11]:
df.title[90]

'香港交易所：一季度32只新股集资1328亿港元 同比增逾8倍'

In [ ]:
########新闻归类系统########
#思路：依次遍历code、名称、省份、行业

In [6]:
#import re
#re.findall('宝钢股份',text[0])
df

,datetime,content,title
0,2021-04-09 23:50:59,【拜登公布首个2022财年1.5万亿美元年度预算提案】当地时间4月9日，美国总统拜登公布了首...,拜登公布首个2022财年1.5万亿美元年度预算提案
1,2021-04-09 23:50:05,【特斯拉在加拿大上架超短续航Model 3】据Electrek报道，特斯拉在加拿大官网上架了...,特斯拉在加拿大上架超短续航Model 3
2,2021-04-09 23:46:46,【欧洲主要股指收盘涨跌不一 德国DAX30指数涨0.26%】欧洲主要股指收盘涨跌不一，德国D...,欧洲主要股指收盘涨跌不一 德国DAX30指数涨0.26%
3,2021-04-09 23:42:55,【美股新能源汽车股短线下挫，特斯拉跌超2%】美股新能源汽车股短线下挫，特斯拉跌超2%，蔚来汽...,美股新能源汽车股短线下挫 特斯拉跌超2%
4,2021-04-09 23:38:07,【在岸人民币兑美元夜盘跌14个基点】在岸人民币兑美元（CNY）北京时间23:30收报6.55...,在岸人民币兑美元夜盘跌14个基点
5,2021-04-09 23:20:22,【猪肉价格连降十周，分析师称还有下跌空间】3月份，新发地市场猪肉价格比较2月份下降11%，同...,猪肉价格连降十周 分析师称还有下跌空间
6,2021-04-09 23:16:53,【铝价一年涨50%，一吨电解铝赚3000元】铝价从去年低点到现在一年时间涨幅超过50%。长江...,铝价一年涨50% 一吨电解铝赚3000元
7,2021-04-09 23:10:55,【南京拟集中推52宗地，起始价867亿元】4月9日，南京披露2020年第一批集中供应地块的信...,南京拟集中推52宗地 起始价867亿元
8,2021-04-09 23:10:11,【国家药监局：祛斑美白化妆品须通过人体功效评价试验】据国家药监局官网9日发布的《化妆品功效宣...,国家药监局：祛斑美白化妆品须通过人体功效评价试验
9,2021-04-09 23:09:07,【麦当劳将关闭沃尔玛内的数百家门店，仅保留150家】据报道，随着越来越多的顾客在线上消费并选...,麦当劳将关闭沃尔玛内的数百家门店 仅保留150家


In [96]:
name_list = ['平安银行',
 '万科A',
 '国华网安',
 '世纪星源',
 '深振业A',
 '全新好','宝钢股份','招行']

In [127]:
def class_news(name_list,content):
    content['count'] = 0
    stock2news = {}
    for name in name_list:
        news_lib = [] 
        for m in range(len(content)):
            try:
                result = re.findall(name.replace('*',''),content['content'][m])
            except Exception as e:
                print(e, name, content['content'][m])
            if len(result) == 0:
                continue 
            else:
                news_lib.append(content['content'][m])
                content['count'][m]=content['count'][m]+1
        stock2news[name]=news_lib
    return stock2news,content
                
        

In [128]:
stock2news,content=class_news(name_list1,df)

<ipython-input-127-6778e037d41c>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  content['count'][m]=content['count'][m]+1


In [134]:
content

,datetime,content,title,count
0,2021-04-09 23:59:00,根据福布斯2021年最新的世界亿万富翁榜单，拥有100位亿万富翁的北京超过拥有99位亿万富翁...,福布斯：北京取代纽约成全球亿万富翁最多城市,0
1,2021-04-09 23:37:00,在岸人民币兑美元（CNY）北京时间23:30收报6.5530元，较周三夜盘收盘跌14点。成交...,在岸人民币兑美元夜盘跌14个基点,0
2,2021-04-09 23:19:00,3月份，新发地市场猪肉价格比较2月份下降11%，同比去年下降29%。据介绍，目前猪肉价格正处...,猪肉价格连降十周 分析师称还有下跌空间,0
3,2021-04-09 23:08:00,据国家药监局官网9日发布的《#化妆品功效宣称评价规范#》，具有祛斑美白、防晒、防脱发、祛痘、...,国家药监局：祛斑美白化妆品须通过人体功效评价试验,0
4,2021-04-09 23:07:00,美国总统拜登寻求将2022财年财政预算提高8.4％至1.52万亿美元，2022财年7690亿...,美国总统拜登寻求将2022财年财政预算提高8.4％至1.52万亿...,0
5,2021-04-09 23:06:00,美国农业部公布报告前，玉米期货价格升至2013年以来最高水平。,美国玉米期货价格升至2013年以来最高水平,0
6,2021-04-09 22:58:00,沪铝主力价格从去年三月底的11000多元的低点，到最近17000多元高位震荡，一年的涨幅超过...,铝价一年涨50% 一吨电解铝赚3000元,1
7,2021-04-09 22:57:00,当地时间9日，世卫组织举行新冠肺炎例行发布会，世卫组织总干事谭德塞表示，目前已有194个国家...,世卫组织：194个国家和经济体已开始接种新冠疫苗,0
8,2021-04-09 22:51:00,顺丰控股、嘉里物流联合公告：双方已获得国家市场监管局有关部分要约及其实施的反垄断批准。,顺丰控股、嘉里物流联合公告：已获得国家市场监管局有关...,1
9,2021-04-09 22:51:00,巴西央行行长内托：今天的通胀数据支持5月份再次加息75个基点。,巴西央行行长内托：今天的通胀数据支持5月份再次加息75个...,0


In [129]:
stock2news

{'平安银行': [],
 '万科A': [],
 '国华网安': [],
 '世纪星源': [],
 '深振业A': [],
 '全新好': [],
 '神州高铁': [],
 '中国宝安': [],
 '美丽生态': [],
 '深物业A': [],
 '南玻A': [],
 '沙河股份': [],
 '深康佳A': [],
 '*ST中华A': [],
 '深粮控股': [],
 '深华发A': [],
 '深科技': [],
 '深天地A': [],
 '特力A': [],
 '飞亚达': [],
 '深圳能源': [],
 '国药一致': [],
 '深深房A': [],
 '富奥股份': [],
 '大悦城': [],
 '深桑达A': [],
 '神州数码': [],
 '中国天楹': [],
 '华联控股': [],
 '深南电A': [],
 '深大通': [],
 '中集集团': [],
 '东旭蓝天': [],
 '中洲控股': [],
 '深纺织A': [],
 '泛海控股': [],
 '京基智农': [],
 '德赛电池': [],
 '深天马A': [],
 '方大集团': [],
 '皇庭国际': [],
 '深赛格': [],
 '华锦股份': [],
 '中金岭南': [],
 '农产品': ['4月9日，农业农村部在京召开视频会议，对加快推进“互联网+”农产品出村进城试点工作进行部署。会议强调，下一步要重点抓好四方面工作。一是抓统筹，要制定工作计划和工作台账，建立试点工作协调机制，把各相关部门力量凝聚起来，扎实推进试点建...',
  '商务部：上周（3月29日至4月4日）全国食用农产品市场价格比前一周下降1.7%。其中猪肉批发价格每公斤34.03元，下降4.7%；鸡蛋批发价格下降0.8%；30种蔬菜平均批发价格下降2.8%；6种水果平均批发价格与前一周持平。',
  '为改变广西壮族自治区农产品加工业起步较晚的不利局面，推动全区农产品加工集聚区高质量发展，自治区农业农村厅日前印发相关方案，将根据各地资源及产业发展优势，认真筛查、梳理、挖掘一批具备创建农产品加工集聚区条件的主体，力争到2021年底...'],
 '深圳华强': [],
 '中兴通讯': ['4月7日，江西省政府与中兴通讯在南昌签署战略合作协议。，双方将在5G网络

In [123]:
name_list1 =['平安银行',
 '万科A',
 '国华网安',
 '世纪星源',
 '深振业A',
 '全新好',
 '神州高铁',
 '中国宝安',
 '美丽生态',
 '深物业A',
 '南玻A',
 '沙河股份',
 '深康佳A',
 '*ST中华A',
 '深粮控股',
 '深华发A',
 '深科技',
 '深天地A',
 '特力A',
 '飞亚达',
 '深圳能源',
 '国药一致',
 '深深房A',
 '富奥股份',
 '大悦城',
 '深桑达A',
 '神州数码',
 '中国天楹',
 '华联控股',
 '深南电A',
 '深大通',
 '中集集团',
 '东旭蓝天',
 '中洲控股',
 '深纺织A',
 '泛海控股',
 '京基智农',
 '德赛电池',
 '深天马A',
 '方大集团',
 '皇庭国际',
 '深赛格',
 '华锦股份',
 '中金岭南',
 '农产品',
 '深圳华强',
 '中兴通讯',
 '北方国际',
 '中国长城',
 '华控赛格',
 '华侨城A',
 '特发信息',
 '海王生物',
 '盐田港',
 '深圳机场',
 '天健集团',
 '广聚能源',
 '中信海直',
 'TCL科技',
 '宜华健康',
 '中成股份',
 '丰原药业',
 '川能动力',
 '华数传媒',
 '中联重科',
 '常山北明',
 '国际实业',
 '申万宏源',
 '东方盛虹',
 '美的集团',
 '潍柴动力',
 '许继电气',
 '冀东水泥',
 '金融街',
 '派林生物',
 '长虹华意',
 '胜利股份',
 '*ST藏格',
 'ST地矿',
 '*ST沈机',
 '英特集团',
 '东旭光电',
 '渤海租赁',
 '民生控股',
 '合肥百货',
 '通程控股',
 '吉林化纤',
 '南京公用',
 'ST宜化',
 '东阿阿胶',
 '徐工机械',
 '兴业矿业',
 '华天酒店',
 '粤高速A',
 '张家界',
 '晨鸣纸业',
 '山东路桥',
 '鄂武商A',
 '绿景控股',
 '国新健康',
 'ST生物',
 '京粮控股',
 '中润资源',
 '珠海港',
 '*ST华塑',
 '新金路',
 '丽珠集团',
 '渝开发',
 '国际医学',
 '荣安地产',
 '四环生物',
 '中兵红箭',
 '长航凤凰',
 '长虹美菱',
 '广州浪奇',
 '岭南控股',
 '红太阳',
 '紫光学大',
 '柳工',
 '广弘控股',
 '冰山冷热',
 '穗恒运A',
 '华金资本',
 '顺钠股份',
 '万泽股份',
 '*ST华映',
 '广宇发展',
 '云南白药',
 '粤电力A',
 '中天金融',
 '佛山照明',
 '皖能电力',
 '中原环保',
 '金浦钛业',
 '金圆股份',
 '航天发展',
 '湖南投资',
 '江铃汽车',
 '创元科技',
 '靖远煤电',
 '安道麦A',
 '泰山石油',
 '神州信息',
 '西部创业',
 '莱茵体育',
 '万向钱潮',
 '我爱我家',
 '烽火电子',
 '陕国投A',
 '*ST大集',
 '渝三峡A',
 '海南海药',
 '海德股份',
 '泸州老窖',
 '苏常柴A',
 '*ST大洲',
 'ST海马',
 '粤宏远A',
 '甘化科工',
 '威孚高科',
 '北部湾港',
 '哈工智能',
 '*ST东电',
 '汇源通信',
 '*ST金洲',
 '贵州轮胎',
 '启迪药业',
 '太阳能',
 '平潭发展',
 '大通燃气',
 '*ST宝实',
 '古井贡酒',
 '东北制药',
 '兴蓉环境',
 '青岛双星',
 '建投能源',
 '韶能股份',
 '盛达资源',
 '渤海股份',
 '顺利办',
 '华媒控股',
 '阳光股份',
 '中迪投资',
 '西安旅游',
 'ST天首',
 '焦作万方',
 '大东海A',
 '奥园美谷',
 '海航投资',
 '中油资本',
 '海螺型材',
 '新华联',
 '恒立实业',
 '吉林敖东',
 '长安汽车',
 '远大控股',
 '天茂集团',
 '高新发展',
 '攀钢钒钛',
 '铜陵有色',
 '顺发恒业',
 '三木集团',
 '合金投资',
 '英力特',
 '风华高科',
 '茂化实华',
 '万方发展',
 '西王食品',
 '仁和药业',
 '格力电器',
 '泰达股份',
 '金岭矿业',
 '金科股份',
 '中钨高新',
 '珠海中富',
 '长春高新',
 '天夏退',
 '*ST永林',
 '湖北广电',
 '经纬纺机',
 '美好置业',
 '荣丰控股',
 '*ST金鸿',
 '阳光城',
 '上峰水泥',
 '*ST当代',
 '智度股份',
 '恒天海龙',
 '襄阳轴承',
 '*ST友谊',
 '山推股份',
 '视觉中国',
 '东方电子',
 '远兴能源',
 '中山公用',
 '东北证券',
 '*ST华讯',
 '国城矿业',
 '宝新能源',
 '亚太实业',
 '惠天热电',
 '滨海能源',
 '炼石航空',
 '沈阳化工',
 '模塑科技',
 '厦门信达',
 '正虹科技',
 '恒逸石化',
 '浙江震元',
 'ST双环',
 '中信特钢',
 '河钢股份',
 '贝瑞基因',
 '京蓝科技',
 '锦龙股份',
 '丰乐种业',
 '中兴商业',
 '黑芝麻',
 '韶钢松山',
 '苏宁环球',
 '中原传媒',
 '新能泰山',
 '西安饮食',
 '湖南发展',
 '美锦能源',
 '京东方A',
 '鲁泰A',
 '华东科技',
 '国元证券',
 '燕京啤酒',
 '四川美丰',
 '泰禾集团',
 '振华科技',
 '罗牛山',
 '中交地产',
 'ST南风',
 '航发控制',
 '普洛药业',
 '国海证券',
 '锌业股份',
 '*ST西发',
 '漳州发展',
 '山西路桥',
 '新华制药',
 '浩物股份',
 '中色股份',
 '中百集团',
 '本钢板材',
 '西藏矿业',
 '通化金马',
 '晋控电力',
 '中航西飞',
 '广发证券',
 '中核科技',
 '新兴铸管',
 '甘咨询',
 '*ST平能',
 '美达股份',
 '长江证券',
 '居然之家',
 '北新建材',
 '北大医药',
 '万年青',
 '华神科技',
 '甘肃电投',
 '华闻集团',
 '英洛华',
 '凯撒旅业',
 '中国武夷',
 '中水渔业',
 '酒鬼酒',
 '一汽解放',
 '四川九洲',
 '北京文化',
 '北清环能',
 '*ST银河',
 '云铝股份',
 '铁岭新城',
 '创维数字',
 '冰轮环境',
 '陕西金叶',
 '德展健康',
 '美利云',
 '智慧农业',
 '航锦科技',
 '岳阳兴长',
 '*ST节能',
 '京山轻机',
 '山东海化',
 '超声电子',
 '太钢不锈',
 '启迪环境',
 '东莞控股',
 '天音控股',
 '鲁西化工',
 '五矿稀土',
 '粤桂股份',
 '*ST长动',
 '富通信息',
 '*ST秦机',
 '财信发展',
 '中信国安',
 '承德露露',
 '华茂股份',
 '高鸿股份',
 '石化机械',
 '冀东装备',
 '五粮液',
 '国风塑业',
 '顺鑫农业',
 '海印股份',
 '银星能源',
 '三湘印象',
 'ST安凯',
 '张裕A',
 '吉电股份',
 '新希望',
 '天山股份',
 '云南铜业',
 '潍柴重机',
 '中广核技',
 '华联股份',
 '湖北能源',
 '城发环境',
 '海南高速',
 '中鼎股份',
 '峨眉山A',
 '中嘉博创',
 '*ST胜尔',
 '欢瑞世纪',
 '亚钾国际',
 '双汇发展',
 '津滨发展',
 '鞍钢股份',
 '赣能股份',
 '现代投资',
 '航天科技',
 '新洋丰',
 '云内动力',
 '厦门港务',
 '浙商中拓',
 '景峰医药',
 '数源科技',
 '大亚圣象',
 '南宁糖业',
 '泸天化',
 '钱江摩托',
 '山大华特',
 '电广传媒',
 '嘉凯城',
 '金陵药业',
 '南方汇通',
 '海信家电',
 '佳电股份',
 '河钢资源',
 '众合科技',
 '福星股份',
 '中国铁物',
 '中钢国际',
 '兰州黄河',
 '中粮科技',
 '中关村',
 '华菱钢铁',
 '神火股份',
 '四川双马',
 '华西股份',
 '冀中能源',
 '紫光股份',
 '南天信息',
 '新乡化纤',
 '重药控股',
 '中国重汽',
 '广济药业',
 '*ST河化',
 '欣龙控股',
 '中通客车',
 '东方能源',
 '首钢股份',
 '锡业股份',
 '中南建设',
 '东方钽业',
 '华东医药',
 '天保基建',
 '长源电力',
 '盈峰环境',
 '蓝焰控股',
 '安泰科技',
 '中科三环',
 '*ST高升',
 'ST中基',
 '佛塑科技',
 '银泰黄金',
 '华铁股份',
 '浪潮信息',
 '桂林旅游',
 '*ST众泰',
 '*ST银亿',
 '*ST中绒',
 '山西焦煤',
 '大庆华科',
 '越秀金控',
 '华工科技',
 '九芝堂',
 '诚志股份',
 '闽东电力',
 '皇台酒业',
 '中国中期',
 '新大陆',
 '隆平高科',
 '华润三九',
 '宗申动力',
 '招商港口',
 '豫能控股',
 '招商积余',
 '招商公路',
 '招商蛇口',
 '新和成',
 '鸿达兴业',
 '伟星股份',
 '华邦健康',
 'ST德豪',
 '精功科技',
 '华兰生物',
 '大族激光',
 '天奇股份',
 '传化智联',
 '盾安环境',
 '凯恩股份',
 '中航机电',
 '永新股份',
 '协鑫能科',
 '世荣兆业',
 '东信和平',
 '亿帆医药',
 '京新药业',
 'ST中捷',
 '科华生物',
 '海特高新',
 '苏宁易购',
 '航天电器',
 '山东威达',
 '分众传媒',
 '思源电气',
 '七匹狼',
 '达安基因',
 '巨轮智能',
 '苏泊尔',
 '丽江股份',
 '旺能环境',
 '华帝股份',
 '联创电子',
 '保利联合',
 '双鹭药业',
 '黔源电力',
 '南京港',
 '登海种业',
 '华孚时尚',
 '兔宝宝',
 '美年健康',
 '国光电器',
 '国机精工',
 '宝鹰股份',
 '宁波华翔',
 '紫光国微',
 '三花智控',
 '中工国际',
 '*ST同洲',
 '云南能投',
 '德美化工',
 '得润电子',
 '横店东磁',
 '中钢天源',
 '*ST威尔',
 '云南旅游',
 '粤水电',
 '浙江交科',
 '宏润建设',
 '远光软件',
 '华峰化学',
 '东华软件',
 '瑞泰科技',
 '景兴纸业',
 '黑猫股份',
 '獐子岛',
 '长城退',
 '*ST凯瑞',
 '软控股份',
 '国轩高科',
 '沙钢股份',
 '*ST雪莱',
 '*ST大港',
 '太阳纸业',
 '苏州固锝',
 '中材科技',
 '金螳螂',
 '万邦德',
 '孚日股份',
 '海鸥住工',
 '万丰奥威',
 '*ST东洋',
 '新野纺织',
 '鲁阳节能',
 'ST新海',
 '金智科技',
 '江苏国泰',
 '中泰化学',
 '国脉科技',
 '青岛金王',
 '生意宝',
 '南岭民爆',
 '山河智能',
 '浔兴股份',
 '海翔药业',
 '天康生物',
 '广东鸿图',
 'ST冠福',
 '广博股份',
 '恒宝股份',
 '信隆健康',
 '莱宝高科',
 '沃华医药',
 '沧州明珠',
 '兴化股份',
 '三钢闽光',
 '威海广泰',
 '三变科技',
 '*ST天润',
 '罗平锌电',
 '三维通信',
 '中国海诚',
 '东港股份',
 '紫鑫药业',
 '康强电子',
 '韵达股份',
 '*ST科陆',
 '*ST天马',
 '梦网科技',
 '天邦股份',
 '湘潭电化',
 '银轮股份',
 '南极电商',
 '露天煤业',
 '中环股份',
 '沃尔核材',
 '利欧股份',
 '恒星科技',
 '广宇集团',
 '天津普林',
 '东南网架',
 '安纳达',
 '实益达',
 '顺络电子',
 '拓邦股份',
 '东华科技',
 '贤丰控股',
 '宁波银行',
 '宏达高科',
 '中核钛白',
 '荣盛发展',
 '*ST新光',
 '北纬科技',
 '西部材料',
 '通润装备',
 '北斗星通',
 '广电运通',
 '石基信息',
 '报喜鸟',
 '湖南黄金',
 '通富微电',
 '正邦科技',
 '汉钟精机',
 '三特索道',
 '常铝股份',
 '远望谷',
 '悦心健康',
 '海南发展',
 '宁波东力',
 '红宝丽',
 '莱茵生物',
 '东方锆业',
 'ST惠程',
 '智光电气',
 '芭田股份',
 '楚江新材',
 '澳洋健康',
 '创新医疗',
 '游族网络',
 'ST东网',
 '*ST江特',
 '御银股份',
 '延华智能',
 '中航光电',
 '纳思达',
 '粤传媒',
 '云海金属',
 '怡亚通',
 '海得控制',
 '华天科技',
 '全聚德',
 '广百股份',
 '*ST巴士',
 '中光学',
 '成飞集成',
 '劲嘉股份',
 '融捷股份',
 '如意集团',
 '武汉凡谷',
 '二三四五',
 '方正电机',
 '证通电子',
 '嘉应制药',
 '东晶电子',
 'ST云投',
 '九鼎新材',
 '金风科技',
 '海亮股份',
 '大连重工',
 '国统股份',
 '海利得',
 '准油股份',
 '合肥城建',
 '达意隆',
 '*ST飞马',
 '宏达新材',
 '天融信',
 '大为股份',
 '大立科技',
 '诺普信',
 '三全食品',
 '合力泰',
 '拓日新能',
 '*ST恒康',
 '东华能源',
 '福晶科技',
 '鱼跃医疗',
 '三力士',
 '濮耐股份',
 '江南化工',
 '奥特迅',
 '合兴包装',
 '鸿博股份',
 '科大讯飞',
 '奥维通信',
 '启明信息',
 '塔牌集团',
 '民和股份',
 '安妮股份',
 '大华股份',
 '恒邦股份',
 '天威视讯',
 '奥特佳',
 '盛新锂能',
 '歌尔股份',
 '九阳股份',
 '力合科创',
 '滨江集团',
 '蔚蓝锂芯',
 '北化股份',
 '*ST聚力',
 '华东数控',
 '大洋电机',
 '联化科技',
 '步步高',
 '上海莱士',
 '川大智胜',
 '泰和新材',
 '*ST海陆',
 '*ST兆新',
 '利尔化学',
 '*ST升达',
 '拓维信息',
 '恩华药业',
 '大东南',
 '新华都',
 '西仪股份',
 '浙富控股',
 '陕天然气',
 '卫士通',
 '美邦服饰',
 '华明装备',
 '东方雨虹',
 '川润股份',
 '水晶光电',
 '华昌化工',
 '桂林三金',
 '万马股份',
 '友阿股份',
 '神开股份',
 '久其软件',
 '*ST联络',
 '光迅科技',
 '博深股份',
 '天润工业',
 '亚太股份',
 '世联行',
 '保龄宝',
 '奇正藏药',
 '超华科技',
 'ST宇顺',
 'ST禾盛',
 '星期六',
 '奥飞娱乐',
 '罗莱生活',
 '信立泰',
 '精艺股份',
 '辉煌科技',
 '博云新材',
 '中电兴发',
 '圣农发展',
 '太阳电缆',
 '齐心集团',
 '西部建设',
 '美盈森',
 '洋河股份',
 '南国置业',
 'ST云网',
 '北新路桥',
 '威创股份',
 '中利集团',
 '东方园林',
 '海大集团',
 '三泰控股',
 '日海智能',
 '南山控股',
 '焦点科技',
 '亚联发展',
 '众生药业',
 '久立特材',
 '乐通股份',
 '海峡股份',
 '华英农业',
 '理工环科',
 '*ST雅博',
 '普利特',
 '洪涛股份',
 '永太科技',
 '富安娜',
 '新朋股份',
 '皇氏集团',
 '得利斯',
 '皖通科技',
 '仙琚制药',
 'ST罗普',
 '英威腾',
 '科华数据',
 'ST人乐',
 '赛象科技',
 '奥普光电',
 '积成电子',
 '格林美',
 '新纶科技',
 '巨力索具',
 '慈文传媒',
 '海宁皮城',
 '潮宏基',
 '柘中股份',
 '泰尔股份',
 '高乐股份',
 '精华制药',
 '北京科锐',
 '漫步者',
 '顺丰控股',
 '杰瑞股份',
 '*ST天娱',
 '兴民智通',
 '*ST赫美',
 '富临运业',
 '森源电气',
 '同德化工',
 '神剑股份',
 '汉王科技',
 '隆基机械',
 '中恒电气',
 '永安药业',
 '台海核电',
 '康力电梯',
 '太极股份',
 '卓翼科技',
 '亚太药业',
 '北方华创',
 '伟星新材',
 '千方科技',
 '中锐股份',
 '亚厦股份',
 '新北洋',
 '国创高新',
 '章源钨业',
 '宏创控股',
 '科远智慧',
 '双箭股份',
 '蓝帆医疗',
 '合众思壮',
 '东山精密',
 '大北农',
 '天原股份',
 '维信诺',
 '新亚制程',
 '航天彩虹',
 '信邦制药',
 '长青股份',
 '北京利尔',
 '力生制药',
 '联发股份',
 '双象股份',
 '星网锐捷',
 '梦洁股份',
 '垒知集团',
 '海普瑞',
 '省广集团',
 '中远海科',
 '和而泰',
 '爱仕达',
 '嘉欣丝绸',
 '四维图新',
 '远东传动',
 '多氟多',
 '齐翔腾达',
 '雅克科技',
 '广联达',
 '延安必康',
 '汉森制药',
 '雷科防务',
 '高德红外',
 '海康威视',
 '爱施德',
 '深南股份',
 '*ST康盛',
 '天虹股份',
 '毅昌股份',
 '达实智能',
 '科伦药业',
 '中粮资本',
 '贵州百灵',
 '凯撒文化',
 '*ST胜利',
 'ST尤夫',
 '云南锗业',
 '兆驰股份',
 '杭氧股份',
 '棕榈股份',
 '九安医疗',
 '太安堂',
 '万里扬',
 '长江健康',
 '兴森科技',
 '誉衡药业',
 '江苏神通',
 '启明星辰',
 '闰土股份',
 '众业达',
 '龙星化工',
 '金洲管道',
 '巨星科技',
 '*ST中南',
 '盛路通信',
 '*ST晨鑫',
 '中原内配',
 '国星光电',
 '摩恩电气',
 '长高集团',
 '华软科技',
 '松芝股份',
 '百川股份',
 '欧菲光',
 '青龙管业',
 '益生股份',
 '晶澳科技',
 '赣锋锂业',
 '珠江啤酒',
 '嘉事堂',
 '沪电股份',
 '众应互联',
 '海格通信',
 '天齐锂业',
 '二六三',
 '申通快递',
 '三维工程',
 '*ST金正',
 '中超控股',
 '双环传动',
 'ST圣莱',
 '榕基软件',
 '立讯精密',
 '宝莫股份',
 '常宝股份',
 '富春环保',
 '新筑股份',
 '双塔食品',
 '广田集团',
 '润邦股份',
 '江海股份',
 '希努尔',
 '嘉麟杰',
 '大金重工',
 '金固股份',
 '浙江永强',
 '山东墨龙',
 '通鼎互联',
 '恒基达鑫',
 '荣盛石化',
 '华斯股份',
 '佳隆股份',
 '*ST辉丰',
 '雅化集团',
 '汉缆股份',
 '*ST科林',
 '山西证券',
 '*ST利源',
 '*ST鼎龙',
 '搜于特',
 '弘高创意',
 '鹏都农牧',
 '协鑫集成',
 '涪陵榨菜',
 '老板电器',
 '天汽模',
 '中顺洁柔',
 '达华智能',
 '*ST蓝丰',
 '宝馨科技',
 '金字火腿',
 '旷达科技',
 '恺英网络',
 '科士达',
 '银河电子',
 '日发精机',
 '齐峰新材',
 '浙江众成',
 '天桥起重',
 '光正眼科',
 '山东矿机',
 '新时达',
 '英飞拓',
 '*ST海源',
 '金财互联',
 '天顺风能',
 '天山铝业',
 '金杯电工',
 '杭锅股份',
 '*ST林重',
 '飞龙股份',
 '海联金汇',
 '司尔特',
 '云图控股',
 '亚太科技',
 '鸿路钢构',
 '中化岩土',
 '万和电气',
 '杰赛科技',
 '东方铁塔',
 '新联电子',
 '春兴精工',
 '金新农',
 '凯美特气',
 '千红制药',
 '尚荣医疗',
 '宝鼎科技',
 '南方轴承',
 '惠博普',
 '三七互娱']

In [117]:
name_list1

['平安银行',
 '万科A',
 '国华网安',
 '世纪星源',
 '深振业A',
 '全新好',
 '神州高铁',
 '中国宝安',
 '美丽生态',
 '深物业A',
 '南玻A',
 '沙河股份',
 '深康佳A',
 '*ST中华A',
 '深粮控股',
 '深华发A',
 '深科技',
 '深天地A',
 '特力A',
 '飞亚达',
 '深圳能源',
 '国药一致',
 '深深房A',
 '富奥股份',
 '大悦城',
 '深桑达A',
 '神州数码',
 '中国天楹',
 '华联控股',
 '深南电A',
 '深大通',
 '中集集团',
 '东旭蓝天',
 '中洲控股',
 '深纺织A',
 '泛海控股',
 '京基智农',
 '德赛电池',
 '深天马A',
 '方大集团',
 '皇庭国际',
 '深赛格',
 '华锦股份',
 '中金岭南',
 '农产品',
 '深圳华强',
 '中兴通讯',
 '北方国际',
 '中国长城',
 '华控赛格',
 '华侨城A',
 '特发信息',
 '海王生物',
 '盐田港',
 '深圳机场',
 '天健集团',
 '广聚能源',
 '中信海直',
 'TCL科技',
 '宜华健康',
 '中成股份',
 '丰原药业',
 '川能动力',
 '华数传媒',
 '中联重科',
 '常山北明',
 '国际实业',
 '申万宏源',
 '东方盛虹',
 '美的集团',
 '潍柴动力',
 '许继电气',
 '冀东水泥',
 '金融街',
 '派林生物',
 '长虹华意',
 '胜利股份',
 '*ST藏格',
 'ST地矿',
 '*ST沈机',
 '英特集团',
 '东旭光电',
 '渤海租赁',
 '民生控股',
 '合肥百货',
 '通程控股',
 '吉林化纤',
 '南京公用',
 'ST宜化',
 '东阿阿胶',
 '徐工机械',
 '兴业矿业',
 '华天酒店',
 '粤高速A',
 '张家界',
 '晨鸣纸业',
 '山东路桥',
 '鄂武商A',
 '绿景控股',
 '国新健康',
 'ST生物',
 '京粮控股',
 '中润资源',
 '珠海港',
 '*ST华塑',
 '新金路',
 '丽珠集团',
 '渝开发',
 '国际医学',
 '荣安地产',
 '四环生物',
 '中兵红箭',
 '